# imports

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# data

# pointers
(set pointers for x_train, x_test, y_train, y_test)

In [227]:
# overwrite when ready.....
x_train = np.zeros((300,2))
x_test  = np.zeros((100,2))
y_train = np.zeros((300,1))
y_test  = np.zeros((100,1))

# define MLP

In [322]:
class AbstractAF:
    """Abstract Activation Function.
    This is an abstract class used to represent
    activation functions in a MultiLayer Perceptitron.

    Each Activation function must have a name
    and implement the following three functions:
    - fw(w,x)         represents a forward pass through the MLP
    - bp(w,x)         represents a dL/dw backprop through the MLP
    - bp_partial(w,x) represents a dL/dh backprop through the MLP <- TODO look into this.....

    This class does not implement any of the three functions.
    Child-classes MUST implement all three functions for 
    backprop to work properly.

    In the current implementation, the following classes are the only valid subclasses:
    - LinearAF
    - ReluAF
    """
    def __init__(self):
        self.name = "Abstract"

    def __repr__(self):
        """Overwrites the representation with class name.
        This function makes the print look cleaner :) 
        """
        return f"<ActivationFunction:{self.name}>"
    
    def fw(self,w,x):
        raise NotImplementedError("Abstract Class cannot run functions.  Please use a subclass.")

    def bp(self,w,x):
        raise NotImplementedError("Abstract Class cannot run functions.  Please use a subclass.")

    def bp_partial(self,w,x):
        # i'll double check the math on this one....
        # from a preliminary check though, i think
        # we might need this to run the backprop cleanly
        # (with this sort of class setup)
        # i think we'd have to chain partial-derivatives
        # or something or other?
        raise NotImplementedError("Abstract Class cannot run functions.  Please use a subclass.")

class LinearAF(AbstractAF):
    """Linear Activation Function"""
    def __init__(self):
        super().__init__()
        self.name = "Linear"
    
    def fw(self,w,x):
        return w.T.dot(x)

    def bp(self,w,x):
        raise NotImplementedError()

class ReluAF(AbstractAF):
    """Relu Activation Function"""
    def __init__(self):
        super().__init__()
        self.name = "Relu"
        
    def fw(self,w,x):
        return np.maximum(0,w.T.dot(x))

    def bp(self,w,x):
        raise NotImplementedError()

In [324]:
class MLP:
    """MultiLayer Perceptron
    Implementation Notes:
    - input and output layers must be defined explicitly.
    """
    def __init__(self):
        self.layers  = []
        self.weights = []

    def add_layer(self,
                  nodes:int,
                  afunc:AbstractAF) -> None:
        """Adds a layer """
        self.layers.append(MLPLayer(nodes,afunc))

    def _init_weights(self) -> None:
        """Initialize weights based on added layers"""
        assert len(self.layers) > 1, "layers must be added"

        # reset weights matrix
        self.weights = []

        # get the shape based on existing layers
        for i in range(1,len(self.layers)):
            w_shape = (self.layers[i-1].get_nodes(),
                       self.layers[i  ].get_nodes())
            self.weights.append(np.zeros(w_shape))

    def fw(self,x:np.array):
        """Performs a forward pass from
        x through n hidden layers to f_w(x)
        by applying an activation function 
        for each layer in the MLP.

        The function also initializes weight
        dimensions, if not done so already.

        Given the input example:
        x_ample = np.ones((3,n))
        
        each column would represent a sample
        ie: 
        > x_ample[:,0]   would be the 1st sample
        > x_ample[:,1]   would be the 2nd sample
        > x_ample[:,n-1] would be the nth sample
        etc.
        
        each row would represent a variable
        ie:
        > x_ample[0,:] would be the 1st parameter
        > x_ample[1,:] would be the 2nd parameter
        > x_ample[2,:] would be the 3rd parameter
        etc.

        The output of this function will generally take the shape:
        (m,n) where n is the number of columns in the input array
        and m is the number of node is the final layer in this MLP.
        In this case, we are predicting one value, how late the
        MBTA will be, and therefore m will always be 1.
        """

        # init weights if not yet done
        if len(self.weights) == 0:
            self._init_weights()

        # loop through and update x iteratively:
        for i in range(1,len(self.layers)):
            x = self.layers[i].fw(self.weights[i-1],x)

        # return x
        return x # <- TODO double check if i should be storing the h's....

    def bp(self,x:np.array,y:np.array):
        raise NotImplementedError()

    def gd(self,x,y,eta:int=10_000):
        # list of errors?
        ls = []
        
        for i in range(eta):
            # self.fw(x)
            # self.bp(x)
            ...

        return ...

In [326]:
class MLPLayer:
    """Represents a single layer in the MLP.
    
    """
    def __init__(self,nodes,afunc):
        self.nodes = int(nodes)
        self.afunc = afunc

    def __repr__(self):
        """overwrite representation for pretty print"""
        return "<MLPLayer: {nodes:"+f"{self.nodes},afunc:{self.afunc}"+"}>"

    def get_nodes(self):
        return self.nodes+0

    def fw(self,w:np.array,x:np.array):
        return self.afunc.fw(w=w,x=x)

    def bp(self,w:np.array,x:np.array):
        return self.afunc.bp(w=w,x=x)

In [328]:
mlp = MLP()
mlp.add_layer(3,LinearAF())  # input x
mlp.add_layer(40,LinearAF()) # hidden layer #1
mlp.add_layer(80,ReluAF())   # hidden layer #2
mlp.add_layer(20,ReluAF())   # hidden layer #3
mlp.add_layer(1,ReluAF())    # prediction f_w(x)
mlp.layers

[<MLPLayer: {nodes:3,afunc:<ActivationFunction:Linear>}>,
 <MLPLayer: {nodes:40,afunc:<ActivationFunction:Linear>}>,
 <MLPLayer: {nodes:80,afunc:<ActivationFunction:Relu>}>,
 <MLPLayer: {nodes:20,afunc:<ActivationFunction:Relu>}>,
 <MLPLayer: {nodes:1,afunc:<ActivationFunction:Relu>}>]

In [330]:
mlp._init_weights()
for weight_matrix in mlp.weights:
    print(weight_matrix.shape)

(3, 40)
(40, 80)
(80, 20)
(20, 1)


In [332]:
# example input:
x_ample = np.ones((3,5))

"""
Given the imput example:
x_ample = np.ones((3,5))

each column would represent a sample
ie: 
> x_ample[:,0] would be the first weather sample
> x_ample[:,1] would be the second weather sample
etc.

each row would represent a variable
ie:
> x_ample[1,:] would be all the tempertures
> x_ample[2,:] would be all the precipitations
> x_ample[3,:] would be all the biases
etc.
"""

# going to print an instance of a forward pass
print(
    mlp.fw(x_ample)
)
"""
mlp.fw(x_ample)
is a matrix that looks like this:
np.array([[0,0,0,0,0]]) # for now:

again, each column would represent
a prediction of y.

If we had multiple rows, as an output,
it could / would be displayed here.
and we could do some sort of 
verification with those.
""";

[[0. 0. 0. 0. 0.]]
